## This is the main notebook file for the machine learning program

In [6]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from Main import read_csv_files

ModuleNotFoundError: No module named 'Main'